In [ ]:
import numpy as np
from scipy.stats import spearmanr
import pandas as pd
import statsmodels.api as sm
from scipy.special import expit
import matplotlib.pyplot as plt

In [ ]:
# generate simulated data
N = 100000
M_X = 10
M_Z = 2
X = np.random.randn(N, M_X)
Z = np.random.randn(N, M_Z)
X_and_Z = np.concatenate([X, Z], axis=1)
beta_T = np.random.randn(M_X + M_Z)
beta_D = 0.2*np.random.randn(M_X + M_Z) + beta_T # related to but non-identical to p(D=1|X)
T = (np.random.random(N,) < expit(np.dot(X_and_Z, beta_T))) * 1.
D = (np.random.random(N,) < expit(np.dot(X_and_Z, beta_D))) * 1.
X_cols = ['X%i' % i for i in range(M_X)]
Z_cols = ['Z%i' % i for i in range(M_Z)]
all_data = pd.concat([pd.DataFrame(X, columns=X_cols), 
                      pd.DataFrame(Z, columns=Z_cols)], 
                     axis=1)
all_data['T'] = T
all_data['D'] = D

# fit the various models we've been discussing
# p(D|X) cannot really be estimated from observed data because we do not observe D for T = 0
D_model = sm.Logit.from_formula('D ~ %s' % '+'.join(X_cols), data=all_data).fit()

# can be estimated from observed data
D_given_T_is_1_model = sm.Logit.from_formula('D ~ %s' % '+'.join(X_cols), 
                                       data=all_data.loc[all_data['T'] == 1]).fit()
T_is_one_model = sm.Logit.from_formula('T ~ %s' % '+'.join(X_cols), data=all_data).fit()

print("Unobservables")
D_given_T_is_0_model = sm.Logit.from_formula('D ~ %s' % '+'.join(X_cols), 
                                       data=all_data.loc[all_data['T'] == 0]).fit() # cannot be estimated from observed data
u_X = D_given_T_is_1_model.predict(all_data) - D_given_T_is_0_model.predict(all_data)
print(u_X.describe())


ground_truth_ps = D_model.predict(all_data)
our_decomposition = D_given_T_is_1_model.predict(all_data) - (1 - T_is_one_model.predict(all_data)) * u_X

plt.figure(figsize=[12, 4], dpi=150)
plt.subplot(1, 3, 1)
plt.scatter(ground_truth_ps, our_decomposition)
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel("p(D=1|X)")
plt.ylabel("p(D=1|T=1, X) - p(T=0|X) * u(X)")
plt.plot([0, 1], [0, 1], color='black', linestyle='--')

plt.subplot(1, 3, 2)
plt.scatter(ground_truth_ps, T_is_one_model.predict(all_data))
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel("p(D=1|X)")
plt.ylabel("p(T=1|X)")
plt.plot([0, 1], [0, 1], color='black', linestyle='--')

plt.subplot(1, 3, 3)
plt.scatter(ground_truth_ps,  D_given_T_is_1_model.predict(all_data))
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel("p(D=1|X)")
plt.ylabel("p(D=1|T=1, X)")
plt.plot([0, 1], [0, 1], color='black', linestyle='--')
plt.show()



